Model creation and loading

In [1]:
import torch
from pathlib import Path
from utils.models.complete_model import create_complete_model, save_complete_model, load_complete_model, save_checkpoint, load_checkpoint


# ALL PATHS
MODELS_DIR = "models/"
SEGMENTER_MODEL_PATH = f"{MODELS_DIR}dino_unet_decoder_finetuned.pth"
save_path = f"{MODELS_DIR}complete_model.pth"
checkpoint_path = f"{MODELS_DIR}model_checkpoint.pth"

# Example usage
device = "cuda" if torch.cuda.is_available() else "cpu"
model = create_complete_model(device=device, SEGMENTER_MODEL_PATH=SEGMENTER_MODEL_PATH)

# Load the model
if Path(save_path).exists():
    model = load_complete_model(model, save_path, device=device, strict=True)

Loaded segmenter weights from models/dino_unet_decoder_finetuned.pth
Loaded complete model weights from models/complete_model.pth


Data loader creation

In [2]:
from utils.data.dataloaders import create_dataloaders

# CheXpert
CHEXPERT_DIR = "Datasets/CheXpertPlus"
chexpert_paths = {
    "chexpert_data_path": f"{CHEXPERT_DIR}/PNG",  # base PNG folder
    "chexpert_data_csv": f"{CHEXPERT_DIR}/df_chexpert_plus_240401_findings.csv",
}

# MIMIC
MIMIC_DIR = "Datasets/MIMIC"
mimic_paths = {
    "mimic_data_path": MIMIC_DIR,
    "mimic_splits_csv": f"{MIMIC_DIR}/mimic-cxr-2.0.0-split.csv.gz",
    "mimic_metadata_csv": f"{MIMIC_DIR}/mimic-cxr-2.0.0-metadata-findings-only.csv",
    "mimic_reports_path": f"{MIMIC_DIR}/cxr-record-list.csv.gz",  # must contain 'path'
    "mimic_images_dir": f"{MIMIC_DIR}/matched_images_and_masks_mimic_224/images",
}

import os
kwargs = {
    # "num_workers": os.cpu_count() // 2 if os.cpu_count() else 4,  # adjust on your VM
    # "persistent_workers": True,           # reuses workers between iterations
    # "prefetch_factor": 4,                 # each worker prefetches batches
    # "pin_memory": True,                   # if using CUDA
    # "drop_last": False
}

train_loader = create_dataloaders(
    chexpert_paths, 
    mimic_paths, 
    batch_size=4,
    split="train", 
    sampling_ratio=0.7,
    **kwargs
)

valid_loader = create_dataloaders(
    chexpert_paths,
    mimic_paths,
    batch_size=4,
    split="valid",
    sampling_ratio=0.7,
    **kwargs
)

images, findings, image_paths, _ = next(iter(train_loader))
print("Batch image tensor shape:", getattr(images, "shape", "N/A"))
print("Batch findings shape:", getattr(findings, "shape", len(findings)))
print("Batch image paths shape:", getattr(image_paths, "shape", len(image_paths)))

Batch image tensor shape: torch.Size([4, 3, 512, 512])
Batch findings shape: 4
Batch image paths shape: 4


Training

In [ ]:
from utils.training import train, EarlyStopping, EarlyStoppingConfig
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
optimizer = Adam(model.parameters(), lr=3e-4, weight_decay=0.1)
# scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=5, min_lr=1e-6)

epochs = (5 * len(train_loader)) // 100
from transformers import get_cosine_schedule_with_warmup
total_steps = 5 * len(train_loader)
warmup_steps = max(1, int(0.05 * total_steps))
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

early = EarlyStopping(EarlyStoppingConfig(
    patience=15, min_delta=1e-4, mode="min", restore_best=True,
    best_ckpt_path="checkpoints/model_best.pth"
))
print(f"Current memory before training: {torch.cuda.memory_allocated(device) / 1e9:.2f} GB") if device == "cuda" else None
torch.cuda.empty_cache() if device == "cuda" else None
train(
    model=model,
    train_loader=train_loader,
    valid_loader=valid_loader,
    optimizer=optimizer,
    epochs=epochs,                       # total target; not "remaining"
    device=device,
    log_dir="runs/chestx_exp1_fixed",       # SAME dir to keep appending
    checkpoint_path="checkpoints/model_epoch.pth",
    validate_every=1,
    ckpt_every=2,
    scheduler=scheduler,
    scheduler_step_on="step",
    early_stopping=early,
    resume_from=None,  # or model_best.pth if you prefer to start from best weights
    # start_epoch=...,                 # optional override
    # start_global_step=...,           # optional override
)


Current memory before training: 0.85 GB
🚀 Starting training from scratch.


Epoch 1 Training:   0%|          | 101/39209 [00:36<3:52:54,  2.80batch/s]


Epoch 1 | Loss: 2.3688 | LR: 0.0000 | Tokens: 30285


Epoch 1 Validation:  33%|███▎      | 101/307 [00:12<00:24,  8.30batch/s]


Epoch 1 | Validation Loss: 2.1451 | Validation Tokens: 30252
New best model found during early stopping.
Saved checkpoint to checkpoints/model_best.pth


...

Epoch 18 | Validation Loss: 4.8317 | Validation Tokens: 28727
Saving periodic checkpoint.
Saved checkpoint to checkpoints/model_epoch.pth


Epoch 19 Training:   0%|          | 101/39209 [00:37<4:00:14,  2.71batch/s]


Epoch 19 | Loss: 3.5807 | LR: 0.0001 | Tokens: 563995


Epoch 19 Validation:  33%|███▎      | 101/307 [00:11<00:23,  8.84batch/s]

Epoch 19 | Validation Loss: 2.7993 | Validation Tokens: 28653
Early stopping triggered at epoch 19. Best epoch: 4 with val loss: 2.103352892398834.
🎉 Training complete.
